# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7faf1cfee580>
├── 'a' --> tensor([[ 1.3460, -0.7432,  2.1855],
│                   [ 0.5057, -1.7111, -0.3922]])
└── 'x' --> <FastTreeValue 0x7faf1cfee6a0>
    └── 'c' --> tensor([[-0.5216, -1.0532, -1.8612,  1.2526],
                        [ 0.5619, -1.6461,  0.9980, -0.1697],
                        [ 0.0157, -1.6163, -0.5815,  1.0017]])

In [4]:
t.a

tensor([[ 1.3460, -0.7432,  2.1855],
        [ 0.5057, -1.7111, -0.3922]])

In [5]:
%timeit t.a

86.9 ns ± 0.384 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7faf1cfee580>
├── 'a' --> tensor([[-0.6503,  0.5648, -1.2979],
│                   [-0.0724, -1.0422,  2.2782]])
└── 'x' --> <FastTreeValue 0x7faf1cfee6a0>
    └── 'c' --> tensor([[-0.5216, -1.0532, -1.8612,  1.2526],
                        [ 0.5619, -1.6461,  0.9980, -0.1697],
                        [ 0.0157, -1.6163, -0.5815,  1.0017]])

In [7]:
%timeit t.a = new_value

86.5 ns ± 0.565 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.3460, -0.7432,  2.1855],
               [ 0.5057, -1.7111, -0.3922]]),
    x: Batch(
           c: tensor([[-0.5216, -1.0532, -1.8612,  1.2526],
                      [ 0.5619, -1.6461,  0.9980, -0.1697],
                      [ 0.0157, -1.6163, -0.5815,  1.0017]]),
       ),
)

In [10]:
b.a

tensor([[ 1.3460, -0.7432,  2.1855],
        [ 0.5057, -1.7111, -0.3922]])

In [11]:
%timeit b.a

76.6 ns ± 0.489 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.5720, -1.2810, -0.4466],
               [ 0.5968, -0.9732,  0.0433]]),
    x: Batch(
           c: tensor([[-0.5216, -1.0532, -1.8612,  1.2526],
                      [ 0.5619, -1.6461,  0.9980, -0.1697],
                      [ 0.0157, -1.6163, -0.5815,  1.0017]]),
       ),
)

In [13]:
%timeit b.a = new_value

653 ns ± 4.37 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.14 µs ± 4.32 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.7 µs ± 202 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

171 µs ± 1.64 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

171 µs ± 2.33 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7faf30017670>
├── 'a' --> tensor([[[ 1.3460, -0.7432,  2.1855],
│                    [ 0.5057, -1.7111, -0.3922]],
│           
│                   [[ 1.3460, -0.7432,  2.1855],
│                    [ 0.5057, -1.7111, -0.3922]],
│           
│                   [[ 1.3460, -0.7432,  2.1855],
│                    [ 0.5057, -1.7111, -0.3922]],
│           
│                   [[ 1.3460, -0.7432,  2.1855],
│                    [ 0.5057, -1.7111, -0.3922]],
│           
│                   [[ 1.3460, -0.7432,  2.1855],
│                    [ 0.5057, -1.7111, -0.3922]],
│           
│                   [[ 1.3460, -0.7432,  2.1855],
│                    [ 0.5057, -1.7111, -0.3922]],
│           
│                   [[ 1.3460, -0.7432,  2.1855],
│                    [ 0.5057, -1.7111, -0.3922]],
│           
│                   [[ 1.3460, -0.7432,  2.1855],
│                    [ 0.5057, -1.7111, -0.3922]]])
└── 'x' --> <FastTreeValue 0x7fae5599b610>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

39.5 µs ± 304 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fae68691d30>
├── 'a' --> tensor([[ 1.3460, -0.7432,  2.1855],
│                   [ 0.5057, -1.7111, -0.3922],
│                   [ 1.3460, -0.7432,  2.1855],
│                   [ 0.5057, -1.7111, -0.3922],
│                   [ 1.3460, -0.7432,  2.1855],
│                   [ 0.5057, -1.7111, -0.3922],
│                   [ 1.3460, -0.7432,  2.1855],
│                   [ 0.5057, -1.7111, -0.3922],
│                   [ 1.3460, -0.7432,  2.1855],
│                   [ 0.5057, -1.7111, -0.3922],
│                   [ 1.3460, -0.7432,  2.1855],
│                   [ 0.5057, -1.7111, -0.3922],
│                   [ 1.3460, -0.7432,  2.1855],
│                   [ 0.5057, -1.7111, -0.3922],
│                   [ 1.3460, -0.7432,  2.1855],
│                   [ 0.5057, -1.7111, -0.3922]])
└── 'x' --> <FastTreeValue 0x7fae68585190>
    └── 'c' --> tensor([[-0.5216, -1.0532, -1.8612,  1.2526],
                        [ 0.5619, -1.6461,  0.9980, -0.1697],
                 

In [23]:
%timeit t_cat(trees)

36.6 µs ± 75.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

78.1 µs ± 284 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.5216, -1.0532, -1.8612,  1.2526],
                       [ 0.5619, -1.6461,  0.9980, -0.1697],
                       [ 0.0157, -1.6163, -0.5815,  1.0017]],
              
                      [[-0.5216, -1.0532, -1.8612,  1.2526],
                       [ 0.5619, -1.6461,  0.9980, -0.1697],
                       [ 0.0157, -1.6163, -0.5815,  1.0017]],
              
                      [[-0.5216, -1.0532, -1.8612,  1.2526],
                       [ 0.5619, -1.6461,  0.9980, -0.1697],
                       [ 0.0157, -1.6163, -0.5815,  1.0017]],
              
                      [[-0.5216, -1.0532, -1.8612,  1.2526],
                       [ 0.5619, -1.6461,  0.9980, -0.1697],
                       [ 0.0157, -1.6163, -0.5815,  1.0017]],
              
                      [[-0.5216, -1.0532, -1.8612,  1.2526],
                       [ 0.5619, -1.6461,  0.9980, -0.1697],
                       [ 0.0157, -1.6163, -0.5815,  1.0017]],

In [26]:
%timeit Batch.stack(batches)

97.7 µs ± 412 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.5216, -1.0532, -1.8612,  1.2526],
                      [ 0.5619, -1.6461,  0.9980, -0.1697],
                      [ 0.0157, -1.6163, -0.5815,  1.0017],
                      [-0.5216, -1.0532, -1.8612,  1.2526],
                      [ 0.5619, -1.6461,  0.9980, -0.1697],
                      [ 0.0157, -1.6163, -0.5815,  1.0017],
                      [-0.5216, -1.0532, -1.8612,  1.2526],
                      [ 0.5619, -1.6461,  0.9980, -0.1697],
                      [ 0.0157, -1.6163, -0.5815,  1.0017],
                      [-0.5216, -1.0532, -1.8612,  1.2526],
                      [ 0.5619, -1.6461,  0.9980, -0.1697],
                      [ 0.0157, -1.6163, -0.5815,  1.0017],
                      [-0.5216, -1.0532, -1.8612,  1.2526],
                      [ 0.5619, -1.6461,  0.9980, -0.1697],
                      [ 0.0157, -1.6163, -0.5815,  1.0017],
                      [-0.5216, -1.0532, -1.8612,  1.2526],
                   

In [28]:
%timeit Batch.cat(batches)

193 µs ± 1.61 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

419 µs ± 8.35 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
